# Orchestrate BigQuery and AutoML tables with Kubeflow pipelines

In [1]:
import kfp
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.notebook


## Create a base image to be used by lightweight components
The image is created by Kaniko Kubernetes service. The image contains the libraries required to interface with BigQuery, Storage and AutoML tables services.

In [2]:
# Configure a staging directory for Kaniko
STAGING_DIR = 'gs://jksandbox/staging'
PROJECT_NAME = 'sandbox-235500'
# Set the base image name
BASE_IMAGE='gcr.io/%s/automltablesbase:dev' % PROJECT_NAME
TARGET_IMAGE='gcr.io/%s/automltablescreate:dev' % PROJECT_NAME

Jupyter docker magic is used to start a Kaniko job. The magic uses a default Kubernetes config.

In [ ]:
%%docker {BASE_IMAGE} {STAGING_DIR}
FROM tensorflow/tensorflow:latest-py3
RUN pip3 install --upgrade pandas
RUN pip3 install --upgrade google-cloud-storage
RUN pip3 install --upgrade google-cloud-automl
RUN pip3 install --upgrade google-cloud-bigquery

### Create python lightweight components

# Load data to Big Query

In [ ]:
from typing import NamedTuple

@dsl.python_component(
    name='automml_create_dataset',
    description='AutoML create dataset',
    base_image=BASE_IMAGE
)
def automl_import_dataset(
    project_id: str,
    dataset_id: str,
    table_id: str) -> NamedTuple('DatasetInfo', 
                               [('name', str), 
                                ('total_rows', int),
                                ('total_columns', int)]):
    
    
    print("Project ID:", project_id)
    print("Dataset ID:", dataset_id)
    print("Table ID:", table_id)
    
    from collections import namedtuple
    result = namedtuple('DatasetInfo', ['name', 'total_rows', 'total_colums'])
    return result('test', 1000, 10)
    

## Create and run a pipeline

### Define a pipeline

In [9]:
from google.cloud import bigquery
import json

@dsl.pipeline(
    name='CLVPipeline',
    description='CLV Pipeline'
)
def clv_pipeline(
    project_id='', 
    dataset_id='', 
    clean_data_query='',
    cleaned_data_table_id='',
    engineer_features_query='',
    features_table_id='',
    target='',
    feature_list='',
    dataset_location='US'
):
    
    BqQueryOp = kfp.components.load_component_from_url(
        'https://raw.githubusercontent.com/kubeflow/pipelines/e8524eefb138725fc06600d1956da0f4dd477178/components/gcp/bigquery/query/component.yaml')

        
    clean_data_op = BqQueryOp(
        query=clean_data_query, 
        project_id=project_id, 
        dataset_id=dataset_id, 
        table_id=cleaned_data_table_id, 
        output_gcs_path='', 
        dataset_location=dataset_location, 
        job_config='').apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    engineer_features_op = BqQueryOp(
        query=engineer_features_query, 
        project_id=project_id, 
        dataset_id=dataset_id, 
        table_id=features_table_id, 
        output_gcs_path='', 
        dataset_location=dataset_location, 
        job_config='').apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    engineer_features_op.after(clean_data_op)
    
    
    
    

### Compile a pipeline

In [10]:


pipeline_func = clv_pipeline
pipeline_filename = pipeline_func.__name__ + '.tar.gz'

kfp.compiler.Compiler().compile(pipeline_func, pipeline_filename)

### Submit the pipeline for execution

In [11]:
threshold_date = '2011-08-08'
predict_date = '2011-12-12'
project_id = 'sandbox-235500'
dataset_id = "CLVDataset"
max_monetary = "15000"

In [12]:
QUERY = '''
SELECT
  customer_id,
  order_date,
  order_value,
  order_qty_articles
FROM
(
  SELECT
    CustomerID AS customer_id,
    PARSE_DATE("%m/%d/%y", SUBSTR(InvoiceDate, 0, 8)) AS order_date,
    ROUND(SUM(UnitPrice * Quantity), 2) AS order_value,
    SUM(Quantity) AS order_qty_articles,
    (
      SELECT
        MAX(PARSE_DATE("%m/%d/%y", SUBSTR(InvoiceDate, 0, 8)))
      FROM
        `<<project_id>>.<<dataset_id>>.data_source` tl
      WHERE
        tl.CustomerID = t.CustomerID
    ) latest_order
  FROM
    `<<project_id>>.<<dataset_id>>.data_source` t
  GROUP BY
      CustomerID,
      order_date
) a

INNER JOIN (
  -- Only customers with more than one positive order values before threshold.
  SELECT
    CustomerID
  FROM (
    -- Customers and how many positive order values  before threshold.
    SELECT
      CustomerID,
      SUM(positive_value) cnt_positive_value
    FROM (
      -- Customer with whether order was positive or not at each date.
      SELECT
        CustomerID,
        (
          CASE
            WHEN SUM(UnitPrice * Quantity) > 0 THEN 1
            ELSE 0
          END ) positive_value
      FROM
        `<<project_id>>.<<dataset_id>>.data_source`
      WHERE
        PARSE_DATE("%m/%d/%y", SUBSTR(InvoiceDate, 0, 8)) < DATE("<<threshold_date>>")
      GROUP BY
        CustomerID,
        SUBSTR(InvoiceDate, 0, 8) )
    GROUP BY
      CustomerID )
  WHERE
    cnt_positive_value > 1
  ) b
ON
  a.customer_id = b. CustomerID
--[START common_clean]
WHERE
  -- Bought in the past 3 months
  DATE_DIFF(DATE("<<predict_date>>"), latest_order, DAY) <= 90
  -- Make sure returns are consistent.
  AND (
    (order_qty_articles > 0 and order_Value > 0) OR
    (order_qty_articles < 0 and order_Value < 0)
  )
'''



clean_data_query = QUERY.replace("<<threshold_date>>", threshold_date)
clean_data_query = clean_data_query.replace("<<predict_date>>", predict_date)
clean_data_query = clean_data_query.replace("<<project_id>>", project_id)
clean_data_query = clean_data_query.replace("<<dataset_id>>", dataset_id)
clean_data_query

'\nSELECT\n  customer_id,\n  order_date,\n  order_value,\n  order_qty_articles\nFROM\n(\n  SELECT\n    CustomerID AS customer_id,\n    PARSE_DATE("%m/%d/%y", SUBSTR(InvoiceDate, 0, 8)) AS order_date,\n    ROUND(SUM(UnitPrice * Quantity), 2) AS order_value,\n    SUM(Quantity) AS order_qty_articles,\n    (\n      SELECT\n        MAX(PARSE_DATE("%m/%d/%y", SUBSTR(InvoiceDate, 0, 8)))\n      FROM\n        `sandbox-235500.CLVDataset.data_source` tl\n      WHERE\n        tl.CustomerID = t.CustomerID\n    ) latest_order\n  FROM\n    `sandbox-235500.CLVDataset.data_source` t\n  GROUP BY\n      CustomerID,\n      order_date\n) a\n\nINNER JOIN (\n  -- Only customers with more than one positive order values before threshold.\n  SELECT\n    CustomerID\n  FROM (\n    -- Customers and how many positive order values  before threshold.\n    SELECT\n      CustomerID,\n      SUM(positive_value) cnt_positive_value\n    FROM (\n      -- Customer with whether order was positive or not at each date.\n      

In [13]:
QUERY = '''
SELECT
  tf.customer_id,
  -- For training period
  -- Copying the calculations from Lifetimes where first orders are ignored
  -- See https://github.com/CamDavidsonPilon/lifetimes/blob/master/lifetimes/utils.py#L246
--[START features_target]
  tf.monetary_dnn,
  tf.cnt_orders AS frequency_dnn,
  tf.cnt_orders - 1 AS frequency_btyd,
  tf.recency,
  tf.T,
  ROUND(tf.recency/cnt_orders, 2) AS time_between,
  ROUND(tf.avg_basket_value, 2) AS avg_basket_value,
  ROUND(tf.avg_basket_size, 2) AS avg_basket_size,
  tf.cnt_returns,
  -- Target calculated for overall period
  ROUND(tt.target_monetary, 2) as target_monetary
--[END features_target]
FROM
  -- This SELECT uses only data before threshold to make features.
  (
    SELECT
      customer_id,
      SUM(order_value) AS monetary_dnn,
      DATE_DIFF(MAX(order_date), MIN(order_date), DAY) AS recency,
      DATE_DIFF(DATE('<<threshold_date>>'), MIN(order_date), DAY) AS T,
      COUNT(DISTINCT order_date) AS cnt_orders,
      AVG(order_qty_articles) avg_basket_size,
      AVG(order_value) avg_basket_value,
      SUM(CASE
          WHEN order_value < 1 THEN 1
          ELSE 0 END) AS cnt_returns
    FROM
      -- Makes the order value = 0 if it is the first one
      (
        SELECT
          a.*,
          (CASE
              WHEN a.order_date = c.order_date_min THEN 0
              ELSE a.order_value END) AS order_value_btyd
        FROM
          `<<project_id>>.<<dataset_id>>.data_cleaned` a
        INNER JOIN (
          SELECT
            customer_id,
            MIN(order_date) AS order_date_min
          FROM
            `<<project_id>>.<<dataset_id>>.data_cleaned`
          GROUP BY
            customer_id) c
        ON
          c.customer_id = a.customer_id
      )
    WHERE
      order_date <= DATE('<<threshold_date>>')
    GROUP BY
      customer_id) tf,

  -- This SELECT uses all records to calculate the target (could also use data after threshold )
  (
    SELECT
      customer_id,
      SUM(order_value) target_monetary
    FROM
      `<<project_id>>.<<dataset_id>>.data_cleaned`
      --WHERE order_date > DATE('<<threshold_date>>')
    GROUP BY
      customer_id) tt
WHERE
  tf.customer_id = tt.customer_id
  AND tf.monetary_dnn > 0
  AND tf.monetary_dnn <= <<max_monetary>>

'''



engineer_features_query = QUERY.replace("<<threshold_date>>", threshold_date)
engineer_features_query = engineer_features_query.replace("<<predict_date>>", predict_date)
engineer_features_query = engineer_features_query.replace("<<project_id>>", project_id)
engineer_features_query = engineer_features_query.replace("<<dataset_id>>", dataset_id)
engineer_features_query = engineer_features_query.replace("<<max_monetary>>", max_monetary)
engineer_features_query

"\nSELECT\n  tf.customer_id,\n  -- For training period\n  -- Copying the calculations from Lifetimes where first orders are ignored\n  -- See https://github.com/CamDavidsonPilon/lifetimes/blob/master/lifetimes/utils.py#L246\n--[START features_target]\n  tf.monetary_dnn,\n  tf.cnt_orders AS frequency_dnn,\n  tf.cnt_orders - 1 AS frequency_btyd,\n  tf.recency,\n  tf.T,\n  ROUND(tf.recency/cnt_orders, 2) AS time_between,\n  ROUND(tf.avg_basket_value, 2) AS avg_basket_value,\n  ROUND(tf.avg_basket_size, 2) AS avg_basket_size,\n  tf.cnt_returns,\n  -- Target calculated for overall period\n  ROUND(tt.target_monetary, 2) as target_monetary\n--[END features_target]\nFROM\n  -- This SELECT uses only data before threshold to make features.\n  (\n    SELECT\n      customer_id,\n      SUM(order_value) AS monetary_dnn,\n      DATE_DIFF(MAX(order_date), MIN(order_date), DAY) AS recency,\n      DATE_DIFF(DATE('2011-08-08'), MIN(order_date), DAY) AS T,\n      COUNT(DISTINCT order_date) AS cnt_orders,\

In [19]:
#Specify pipeline argument values
CLEANED_DATA_TABLE_ID = 'data_cleaned'
FEATURES_TABLE_ID = 'features'
TARGET_COLUMN_NAME = 'monetary_target'
FEATURE_LIST = []

arguments = {
    'project_id': project_id,
    'dataset_id': dataset_id,
    'clean_data_query': clean_data_query,
    'cleaned_data_table_id': CLEANED_DATA_TABLE_ID,
    'engineer_features_query': engineer_features_query,
    'features_table_id': FEATURES_TABLE_ID,
    'target': TARGET_COLUMN_NAME,
    'feature_list': FEATURE_LIST,
}


HOST = 'http://localhost:8082/api/v1/namespaces/kubeflow/services/ml-pipeline:8888/proxy'
EXPERIMENT_NAME = 'CLV_TRAINING'

client = kfp.Client(HOST)
experiment = client.create_experiment(EXPERIMENT_NAME)

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)
print(run_result)

{'created_at': datetime.datetime(2019, 5, 6, 3, 31, 2, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'finished_at': None,
 'id': '5f502bd3-6faf-11e9-b85b-42010a80003c',
 'metrics': None,
 'name': 'clv_pipeline run',
 'pipeline_spec': {'parameters': [{'name': 'feature-list', 'value': '[]'},
                                  {'name': 'clean-data-query',
                                   'value': '\n'
                                            'SELECT\n'
                                            '  customer_id,\n'
                                            '  order_date,\n'
                                            '  order_value,\n'
                                            '  order_qty_articles\n'
                                            'FROM\n'
                                            '(\n'
                                            '  SELECT\n'
                                            '    CustomerID AS customer_id,\n'
                                  